## Lokasi Dataset : [dataset](https://drive.google.com/drive/folders/1A3TpCqabeUsYbd5NT_DDlI64HkW9GIg_?usp=drive_link)

## Import Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# CNN

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = '/content/drive/MyDrive/model'

categories = os.listdir(data_dir)
categories.sort()

print("Kategori Isyarat Tangan:", categories)


Kategori Isyarat Tangan: ['kamu', 'nama', 'siapa']


In [ ]:
image_size = 224
X = []  # Data gambar
y = []  # Label kategori

for label, category in enumerate(categories):
    category_path = os.path.join(data_dir, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) ubah warna ke RGB
        img = cv2.resize(img, (image_size, image_size)) #resize gmb
        X.append(img)
        y.append(label)

# Konvert ke np array
X = np.array(X)
y = np.array(y)

# Normalisasi image
X = X / 255.0

In [ ]:
# bagi dataset menjadi training dan testing (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#ubah label ke format one-hot encoding
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=len(categories))
y_test = to_categorical(y_test, num_classes=len(categories))

In [ ]:
# Membangun model CNN
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(categories), activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.4475 - loss: 1.0607 - val_accuracy: 0.9000 - val_loss: 0.2151
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.9556 - loss: 0.1810 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 4.0531e-07
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - accuracy: 1.0000 - loss: 5.8902e-04 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.9915 - loss: 0.0311 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 1.0000 - loss: 1.2945e-04 - val_accuracy: 1.0000 - val_loss: 1.8636e-06
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.9915 - loss: 0.0153 - val_accuracy: 1.0000 - val_loss: 4.6491e-07
Epoch 8/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 1.0000 - loss: 1.2547e-04 - val_accuracy: 1.

In [ ]:
# Evaluasi model pada data testing
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 1.1921e-08
Test Loss: 1.192092646817855e-08
Test Accuracy: 1.0


In [ ]:
# Contoh prediksinya
test_img = cv2.imread('/content/drive/MyDrive/model/kamu/kamu_20.jpg')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_img = cv2.resize(test_img, (image_size, image_size))
test_img = np.expand_dims(test_img, axis=0)  # Menambah dimensi batch

# Normalisasi gambar
test_img = test_img / 255.0

# Prediksi kategori gambar
pred = model.predict(test_img)
predicted_class = np.argmax(pred)
print("Prediksi Kategori:", categories[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Prediksi Kategori: kamu


# INTEGRASI CCN + LSTM

In [ ]:
import numpy as np
import cv2
import os

image_size = 224
sequence_length = 10  #Panjang urutan gambarnya per video
X = []  # Data gambar
y = []  # Label kategori

# Membaca data gambar urutan
data_dir = '/content/drive/MyDrive/model'
categories = os.listdir(data_dir)
categories.sort()

for label, category in enumerate(categories):
    category_path = os.path.join(data_dir, category)

    #ngambil file gambar dari folder
    files = os.listdir(category_path)
    for i in range(0, len(files) - sequence_length + 1, sequence_length):
        sequence_images = []
        for j in range(sequence_length):
            img_path = os.path.join(category_path, files[i + j])
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  #koversi ke RGB gambarnya
            img = cv2.resize(img, (image_size, image_size)) #ubah ukuran gmbr
            sequence_images.append(img)

        X.append(np.array(sequence_images))
        y.append(label)

# Konversi ke array
X = np.array(X)
y = np.array(y)

#Normalisasi
X = X / 255.0

# Mengubah label ke one-hot encoding
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=len(categories))

In [ ]:
from sklearn.model_selection import train_test_split

# Membagi dataset menjadi training dan testing (80% untuk training, 20% untuk testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout, Input

#Integrasi CNN & LSTM
model = Sequential()

model.add(Input(shape=(sequence_length, image_size, image_size, 3)))

model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

# LSTM-nya
model.add(LSTM(128, activation='relu', return_sequences=False))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(categories), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_21                  │ (None, 10, 222, 222, 32)    │             896 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_22                  │ (None, 10, 111, 111, 32)    │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_23                  │ (None, 10, 109, 109, 64)    │          18,496 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_24                  │ (None, 10, 54, 54, 64)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_25                  │ (None, 10, 52, 52, 128)     │          73,856 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_26                  │ (None, 10, 26, 26, 128)     │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_27                  │ (None, 10, 86528)           │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │      44,368,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,478,531 (169.67 MB)

 Trainable params: 44,478,531 (169.67 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Melatih model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 52s 52s/step - accuracy: 0.5000 - loss: 1.0958 - val_accuracy: 0.6667 - val_loss: 2.3544
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 73s 73s/step - accuracy: 0.5000 - loss: 10.4324 - val_accuracy: 0.3333 - val_loss: 1.6924
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 0.5833 - loss: 3.3241 - val_accuracy: 0.3333 - val_loss: 1.0320
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step - accuracy: 0.4167 - loss: 2.1420 - val_accuracy: 0.6667 - val_loss: 0.6233
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 37s 37s/step - accuracy: 0.7500 - loss: 0.6624 - val_accuracy: 0.6667 - val_loss: 0.6663
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step - accuracy: 0.8333 - loss: 0.7501 - val_accuracy: 1.0000 - val_loss: 0.5567
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step - accuracy: 0.9167 - loss: 0.5859 - val_accuracy: 0.6667 - val_loss: 0.3565
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step - accuracy: 0.8333 - loss: 0.3713 - val_accuracy: 1.0000 - val_loss: 0.1162

In [ ]:
# Evaluasi model pada data testing
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 2.7418e-06
Test Loss: 2.741804564720951e-06
Test Accuracy: 1.0


In [ ]:
# Contoh Prediksi
test_sequence = np.expand_dims(X_test[0], axis=0)  # diambil dari data test
test_sequence = test_sequence.astype(np.float32)

# Prediksi kategori
pred = model.predict(test_sequence)
predicted_class = np.argmax(pred)

print("Prediksi Kategori:", categories[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediksi Kategori: nama


In [ ]:
#save model

In [ ]:
#kovert ke tflite